In [1]:
%load_ext autoreload
%autoreload 2

Naszym zadaniem będzie napisanie prostego programu, który pozwoli na kompresję obrazów przy pomocy dyskretnej transformaty cosinusowej. 

Funkcje programu powinny być następujące:

> program wykonywalny (dctcompress.py), który korzysta z biblioteki argparse i rozpoznaje następujące argumenty (plik wejsciowy: .bmp lub .dct), współczynnik kompresji --ratio (typu float) (2 pkt)<br>

> Kiedy program dostanie plik wejściowy w postaci pliku BMP i współczynnik kompresji (domyślnie 0.5), powinien dokonać transformaty kosinusowej danych obrazu i następnie zachować tylko tyle współczynników widma obrazu, aby po ich zapisaniu do pliku otrzymać współczynnik kompresji nie gorszy od zadanego, ale możliwie mu bliski. Oczywiście należy także zachować dodtkowe informacje, aby możliwa była dekompresja (m.in. rozmiar oryginalnego obrazu). Obraz wynikowy powinien zostać zapisany do pliku o tej samej nazwie i rozszerzeniu .dct (5 pkt))<br>

> Kiedy program dostanie plik wejściowy w postaci pliku .dct, powiniew wczytać zawarte w nim dane (tj. dane obrazu i zachowane współczynniki widma), a następnie wpisać je do macierzy, uzupełnić zerami i dokonać odwrotnej transformaty kosinusowej i zapisać wynikowy obraz do pliku .bmp (5 pkt). )<br>

> Wersja podstawowa programu powinna działać dla obrazów w skali szarości (8 bitów/pixel), podobnie do tego co robiliśmy na zajęciach dotyczących kompresji, natomiast wersja pełna, powinna również działać na obrazach w formacie RGB (3*8bitów/pixel), traktując każdą ze składowych(RGB) jako osobną macierz, która podlega kompresji. (3 pkt))<br>

> Jako rozwiązanie należy nadesłać plik dctcompress.py zawierający kod rozwiązania wyposażony w czytelne komentarze. 

Dodatkowa, funkcjonalność (bonusowe 3 punkty):

jeśli podczas kompresji podamy programowi flagę --gzip, program wykorzystuje kompresję Lempela-Ziv'a w module gzip, aby w pliku o zadanym rozmiarze przechować więcej współczynników. Należy wtedy (metodą prób i błędów) dobrać liczbę współczynników tak, aby po kompresji gzip rozmiar pliku nadal nie przekraczał zadanego współczynnika kompresji, ale był mu bliski. Warto zastanowić sie nad tym jak dobierać liczbę współczynników. Należy spodziewać się dość równomiernego współczynnika kompresji dla zestawu liczb.

In [2]:
import numpy as np
from PIL import Image

In [3]:
img = Image.open('DNA-monument.bmp')
img.load()

In [12]:
from dctcompress import bmp2dct, dct2bmp

In [16]:
bmp2dct('DNA-monument-BW', 0.5)

(500, 332)


KeyboardInterrupt: 

In [6]:
dct2bmp('DNA-monument-BW')

In [ ]:
import numpy as np
from PIL import Image
from scipy import fftpack

In [ ]:
def get_2D_dct(img):
    """ Get 2D Cosine Transform of Image
    """
    return fftpack.dct(fftpack.dct(img.T, norm='ortho').T, norm='ortho')

def get_2d_idct(coefficients):
    """ Get 2D Inverse Cosine Transform of Image
    """
    return fftpack.idct(fftpack.idct(coefficients.T, norm='ortho').T, norm='ortho')

def save_img(img, img_name):
    im = Image.fromarray(img)
    if im.mode != 'RGB':
        im = im.convert('RGB')
    im.save(img_name)

In [ ]:
def bmp2dct(filename, ratio, eps=0.05):
    im = Image.open(f'{filename}.bmp', mode='r')
    img = np.array(im, dtype=float)
    a = 0
    b = img.shape[0]
    curr_ratio = 1.0
    save_img(img, f'{filename}_0.bmp')
    coefs = get_2D_dct(img)
    coefs_p = np.zeros_like(coefs)
    while curr_ratio < ratio or curr_ratio > ratio + eps:
        del coefs_p
        print(curr_ratio, a, b)
        coefs_p = np.zeros_like(coefs)
        i = (a + b) // 2
        coefs_p[:i, :i] = coefs[:i, :i]
        save_img(get_2d_idct(coefs_p), f'{filename}_1.bmp')
        curr_ratio = os.path.getsize(f'{filename}_1.bmp') / os.path.getsize(f'{filename}_0.bmp')
        print(os.path.getsize(f'{filename}_1.bmp'))
        if curr_ratio > ratio:
            b = i
        else:
            a = i
    data = (coefs_p[:i, :i], coefs_p.shape)
    with open(f'{filename}.dct', 'wb') as f:
        pickle.dump(data, f)

In [ ]:
bmp2dct(filename, 0.5)